<a href="https://colab.research.google.com/github/Cumberkid/Learning-the-Optimal-Solution-Path/blob/main/experiments/fair-regression/mini-experiments/Class_Imbalance_%26_Fairness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

# Bar Passage

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Experiments/bar_pass_prediction.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22407 entries, 0 to 22406
Data columns (total 39 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   decile1b                 20803 non-null  float64
 1   decile3                  20803 non-null  float64
 2   ID                       22407 non-null  int64  
 3   decile1                  21315 non-null  float64
 4   sex                      22402 non-null  float64
 5   race                     22391 non-null  float64
 6   cluster                  22311 non-null  float64
 7   lsat                     22407 non-null  float64
 8   ugpa                     22407 non-null  float64
 9   zfygpa                   21423 non-null  float64
 10  DOB_yr                   22357 non-null  float64
 11  grad                     22404 non-null  object 
 12  zgpa                     21118 non-null  float64
 13  bar1                     22407 non-null  object 
 14  bar1_yr               

In [ ]:
print("Feature Names:", ", ".join(df.columns))

Feature Names: decile1b, decile3, ID, decile1, sex, race, cluster, lsat, ugpa, zfygpa, DOB_yr, grad, zgpa, bar1, bar1_yr, bar2, bar2_yr, fulltime, fam_inc, age, gender, parttime, male, race1, race2, Dropout, other, asian, black, hisp, pass_bar, bar, bar_passed, tier, index6040, indxgrp, indxgrp2, dnn_bar_pass_prediction, gpa


In [ ]:
df = df.dropna()
len(df)

20427

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Assuming 'categorical_column' is a column that you want to one-hot encode
# Replace 'categorical_column' with the actual name of the categorical column
categorical_column = ['grad', 'gender', 'race1', 'race2', 'Dropout', 'indxgrp', 'indxgrp2']

# Create a DataFrame with the categorical column
categorical_df = df[categorical_column]

# Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, drop='first')

# Fit and transform the categorical column
one_hot_encoded = encoder.fit_transform(categorical_df)

# Convert the one-hot encoded result to a DataFrame
one_hot_encoded_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_column))


# Concatenate the one-hot encoded DataFrame with the original DataFrame, dropping the original categorical column
df_encoded = pd.concat([df.drop(columns=categorical_column), one_hot_encoded_df], axis=1)

df_encoded = df_encoded.dropna()

print(df_encoded.head())


   decile1b  decile3    ID  decile1  sex  race  cluster  lsat  ugpa  zfygpa  \
0      10.0     10.0   2.0     10.0  1.0   7.0      1.0  44.0   3.5    1.33   
1       5.0      4.0   3.0      5.0  1.0   7.0      2.0  29.0   3.5   -0.11   
2       3.0      2.0  36.0      3.0  2.0   7.0      3.0  36.0   3.5   -0.64   
3       7.0      4.0  52.0      7.0  2.0   7.0      3.0  39.0   3.5    0.34   
4       9.0      8.0  55.0      9.0  2.0   7.0      4.0  48.0   3.5    1.02   

   ...  indxgrp_f 640-700  indxgrp_g 700+ indxgrp2_b 400-460  \
0  ...                0.0             1.0                0.0   
1  ...                1.0             0.0                0.0   
2  ...                0.0             1.0                0.0   
3  ...                0.0             1.0                0.0   
4  ...                0.0             1.0                0.0   

   indxgrp2_c 460-520 indxgrp2_d 520-580  indxgrp2_e 580-640  \
0                 0.0                0.0                 0.0   
1           

In [ ]:
len(df_encoded)

18668

In [ ]:
print("Feature Names:", ", ".join(df_encoded.columns))

Feature Names: decile1b, decile3, ID, decile1, sex, race, cluster, lsat, ugpa, zfygpa, DOB_yr, zgpa, bar1, bar1_yr, bar2, bar2_yr, fulltime, fam_inc, age, parttime, male, other, asian, black, hisp, pass_bar, bar, bar_passed, tier, index6040, dnn_bar_pass_prediction, gpa, gender_male, race1_black, race1_hisp, race1_other, race1_white, race2_b white, race2_c other, indxgrp_b 400-460, indxgrp_c 460-520, indxgrp_d 520-580, indxgrp_e 580-640, indxgrp_f 640-700, indxgrp_g 700+, indxgrp2_b 400-460, indxgrp2_c 460-520, indxgrp2_d 520-580, indxgrp2_e 580-640, indxgrp2_f 640-700, indxgrp2_g 700-760, indxgrp2_h 760-820, indxgrp2_i 820+


In [ ]:
# Choose the features to be used
cols = ['decile1b', 'decile3', 'ID', 'decile1', 'sex', 'race', 'cluster', 'lsat',
        'ugpa', 'zfygpa', 'DOB_yr', 'zgpa', 'fulltime', 'fam_inc', 'age', 'parttime',
        'male', 'other', 'asian', 'black', 'hisp', 'tier', 'index6040', 'gpa',
        'gender_male', 'race1_black', 'race1_hisp', 'race1_other', 'race1_white',
        'race2_b white', 'race2_c other', 'indxgrp_b 400-460', 'indxgrp_c 460-520',
        'indxgrp_d 520-580', 'indxgrp_e 580-640', 'indxgrp_f 640-700', 'indxgrp_g 700+',
        'indxgrp2_b 400-460', 'indxgrp2_c 460-520', 'indxgrp2_d 520-580', 'indxgrp2_e 580-640',
        'indxgrp2_f 640-700', 'indxgrp2_g 700-760', 'indxgrp2_h 760-820', 'indxgrp2_i 820+']

## no sub-sampling

In [ ]:
X = df_encoded[cols]
y = df_encoded['pass_bar']

In [ ]:
# preprocess data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# scaler = RobustScaler()
scaler.fit(X)
X_encoded = scaler.transform(X)

model = LogisticRegression(solver='lbfgs', max_iter=10000)

# Fit the model
model.fit(X, y)

LogisticRegression(max_iter=10000)

In [ ]:
df_white = df_encoded[df_encoded['race'] == 7]
df_other = df_encoded[df_encoded['race'] != 7]

len(df_white), len(df_other)

(15755, 2913)

In [ ]:
X_white = df_white[cols]
y_white = df_white['pass_bar']

In [ ]:
accuracy_score(y_white, model.predict(X_white))

0.9677562678514757

In [ ]:
X_other = df_other[cols]
y_other = df_other['pass_bar']

In [ ]:
accuracy_score(y_other, model.predict(X_other))

0.870236869207003

In [ ]:
from sklearn.metrics import confusion_matrix
?confusion_matrix

In [ ]:
df_pass = df_encoded[df_encoded['pass_bar'] == 1]
df_fail = df_encoded[df_encoded['pass_bar'] == 0]

len(df_pass), len(df_fail)

(17774, 894)

In [ ]:
cm_white = confusion_matrix(y_white, model.predict(X_white))
cm_white

array([[    4,   506],
       [    2, 15243]])

In [ ]:
TPR_white =  cm_white.ravel()[3] / (cm_white.ravel()[3] + cm_white.ravel()[2] )                               #TPR = TP / P = TP / (TP + FN)
print('TPR is: %.4f' % TPR_white)

FPR_white = cm_white.ravel()[1] / (cm_white.ravel()[1] + cm_white.ravel()[0] )                         #FPR = FP / N = FP / (FP + TN)
print('FPR is: %.4f' % FPR_white)

TPR is: 0.9999
FPR is: 0.9922


In [ ]:
cm_other = confusion_matrix(y_other, model.predict(X_other))
cm_other

array([[  28,  356],
       [  22, 2507]])

In [ ]:
TPR_other =  cm_other.ravel()[3] / (cm_other.ravel()[3] + cm_other.ravel()[2] )                               #TPR = TP / P = TP / (TP + FN)
print('TPR is: %.4f' % TPR_white)

FPR_other = cm_other.ravel()[1] / (cm_other.ravel()[1] + cm_other.ravel()[0] )                         #FPR = FP / N = FP / (FP + TN)
print('FPR is: %.4f' % FPR_white)

TPR is: 0.9999
FPR is: 0.9922


## sub-sampling

### no-bias sampling

In [ ]:
sp = df_encoded.sample(n=1000, random_state=42)

In [ ]:
sp_white = sp[sp['race'] == 7]
sp_other = sp[sp['race'] != 7]

len(sp_white), len(sp_other)

(848, 152)

In [ ]:
passed = sp[sp['pass_bar'] == 1]
len(passed)

959

In [ ]:
X = sp[cols]
y = sp['pass_bar']

In [ ]:
# preprocess data
from sklearn.preprocessing import StandardScaler, RobustScaler

scaler = StandardScaler()
# scaler = RobustScaler()
scaler.fit(X)
X = scaler.transform(X)

In [ ]:
clf = LogisticRegression(solver='lbfgs', max_iter=100).fit(X, y)

In [ ]:
X_white = sp_white[cols]
y_white = sp_white['pass_bar']
X_other = sp_other[cols]
y_other = sp_other['pass_bar']

X_white.shape

(848, 45)

In [ ]:
accuracy_score(y_white, clf.predict(X_white))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


0.972877358490566

In [ ]:
accuracy_score(y_other, clf.predict(X_other))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


0.881578947368421

In [ ]:
from sklearn.metrics import confusion_matrix

?confusion_matrix

In [ ]:
cm_white = confusion_matrix(y_white, clf.predict(X_white))
cm_white

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


array([[  0,  23],
       [  0, 825]])

In [ ]:
TPR_white =  cm_white.ravel()[3] / (cm_white.ravel()[3] + cm_white.ravel()[2] )                               #TPR = TP / P = TP / (TP + FN)
print('TPR is: %.4f' % TPR_white)

FPR_white = cm_white.ravel()[1] / (cm_white.ravel()[1] + cm_white.ravel()[0] )                         #FPR = FP / N = FP / (FP + TN)
print('FPR is: %.4f' % FPR_white)

TPR is: 1.0000
FPR is: 1.0000


In [ ]:
cm_other = confusion_matrix(y_other, clf.predict(X_other))
cm_other

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


array([[  0,  18],
       [  0, 134]])

In [ ]:
TPR_other =  cm_other.ravel()[3] / (cm_other.ravel()[3] + cm_other.ravel()[2] )                               #TPR = TP / P = TP / (TP + FN)
print('TPR is: %.4f' % TPR_white)

FPR_other = cm_other.ravel()[1] / (cm_other.ravel()[1] + cm_other.ravel()[0] )                         #FPR = FP / N = FP / (FP + TN)
print('FPR is: %.4f' % FPR_white)

TPR is: 1.0000
FPR is: 1.0000


### biased sampling

In [ ]:
sp_white = df_white.sample(n=950, random_state=42)
sp_other = df_other.sample(n=50, random_state=42)

len(sp_white), len(sp_other)

(950, 50)

In [ ]:
df_white = df[df['race1'] == 'white']
df_other = df[df['race1'] != 'white']

len(df_white), len(df_other)

(17139, 3288)

In [ ]:
sp_df = pd.concat([sp_white, sp_other])
sp_df

,decile1b,decile3,ID,decile1,sex,race,cluster,lsat,ugpa,zfygpa,...,hisp,pass_bar,bar,bar_passed,tier,index6040,indxgrp,indxgrp2,dnn_bar_pass_prediction,gpa
21077,10.0,9.0,16729,10.0,2.0,7.0,1.0,37.0,3.8,1.40,...,0,1,a Passed 1st time,True,4.0,806.315771,g 700+,h 760-820,0.999698,3.8
9705,9.0,10.0,17712,9.0,1.0,7.0,3.0,34.0,3.6,1.22,...,0,1,a Passed 1st time,True,3.0,738.947352,g 700+,g 700-760,0.999319,3.6
13897,9.0,9.0,5099,9.0,2.0,7.0,6.0,35.0,3.7,1.10,...,0,1,a Passed 1st time,True,1.0,764.736825,g 700+,h 760-820,0.983144,3.7
345,6.0,2.0,4990,6.0,2.0,7.0,3.0,37.0,3.5,0.11,...,0,1,b Passed 2nd time,True,3.0,776.315771,g 700+,h 760-820,0.979804,3.5
3502,10.0,10.0,20619,10.0,1.0,7.0,5.0,42.0,3.4,1.75,...,0,1,a Passed 1st time,True,6.0,845.263136,g 700+,i 820+,0.998464,3.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7886,5.0,2.0,23467,6.0,2.0,6.0,4.0,37.0,3.2,0.05,...,0,1,a Passed 1st time,True,5.0,746.315771,g 700+,g 700-760,0.996526,3.2
6131,4.0,4.0,24886,4.0,1.0,3.0,6.0,28.5,2.2,-0.31,...,0,0,c Failed,False,1.0,512.105251,c 460-520,c 460-520,0.890770,2.2
442,3.0,3.0,6424,4.0,1.0,8.0,1.0,30.0,3.5,-0.47,...,0,1,a Passed 1st time,True,4.0,665.789460,f 640-700,f 640-700,0.979804,3.5
21007,7.0,8.0,14850,7.0,2.0,8.0,3.0,26.5,3.8,0.35,...,0,1,a Passed 1st time,True,3.0,640.526305,f 640-700,f 640-700,0.999698,3.8


In [ ]:
passed = sp_df[sp_df['pass_bar'] == 1]
len(passed)

960

In [ ]:
grad = sp_df[sp_df['grad'] == 'Y']
len(grad)

1000

In [ ]:
# Choose the features to be used
cols = ['tier', 'race', 'sex', 'ugpa', 'lsat', 'fulltime', 'fam_inc']
X = sp_df[cols]
y = sp_df['pass_bar']

In [ ]:
# preprocess data
from sklearn.preprocessing import StandardScaler, RobustScaler

scaler = StandardScaler()
# scaler = RobustScaler()
scaler.fit(X)
X = scaler.transform(X)

In [ ]:
clf = LogisticRegression(solver='lbfgs', max_iter=100).fit(X, y)

In [ ]:
X_white = sp_white[cols]
y_white = sp_white['pass_bar']
X_other = sp_other[cols]
y_other = sp_other['pass_bar']

X_white.shape

(950, 7)

In [ ]:
accuracy_score(y_white, clf.predict(X_white))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


0.9673684210526315

In [ ]:
accuracy_score(y_other, clf.predict(X_other))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


0.82

In [ ]:
from sklearn.metrics import confusion_matrix

?confusion_matrix

In [ ]:
cm_white = confusion_matrix(y_white, clf.predict(X_white))
cm_white

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


array([[  0,  31],
       [  0, 919]])

In [ ]:
TPR_white =  cm_white.ravel()[3] / (cm_white.ravel()[3] + cm_white.ravel()[2] )                               #TPR = TP / P = TP / (TP + FN)
print('TPR is: %.4f' % TPR_white)

FPR_white = cm_white.ravel()[1] / (cm_white.ravel()[1] + cm_white.ravel()[0] )                         #FPR = FP / N = FP / (FP + TN)
print('FPR is: %.4f' % FPR_white)

TPR is: 1.0000
FPR is: 1.0000


In [ ]:
cm_other = confusion_matrix(y_other, clf.predict(X_other))
cm_other

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


array([[ 0,  9],
       [ 0, 41]])

In [ ]:
TPR_other =  cm_other.ravel()[3] / (cm_other.ravel()[3] + cm_other.ravel()[2] )                               #TPR = TP / P = TP / (TP + FN)
print('TPR is: %.4f' % TPR_white)

FPR_other = cm_other.ravel()[1] / (cm_other.ravel()[1] + cm_other.ravel()[0] )                         #FPR = FP / N = FP / (FP + TN)
print('FPR is: %.4f' % FPR_white)

TPR is: 1.0000
FPR is: 1.0000


# Breast Cancer

In [ ]:
from sklearn.datasets import load_breast_cancer

In [ ]:
# load breast cancer data
X, y = load_breast_cancer(return_X_y=True)

In [ ]:
# preprocess data
from sklearn.preprocessing import StandardScaler, RobustScaler

scaler = StandardScaler()
# scaler = RobustScaler()
scaler.fit(X)
X = scaler.transform(X)

In [ ]:
df = pd.DataFrame(X).describe()

df

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
count,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,...,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02
mean,-3.153111e-15,-6.568462e-15,-6.993039e-16,-8.553985e-16,6.081447e-15,-1.136369e-15,-2.997017e-16,1.023981e-15,-1.860648e-15,-1.504752e-15,...,-2.297713e-15,1.742016e-15,-1.198807e-15,6.118909e-16,-5.094929e-15,-2.122887e-15,6.118909e-16,-1.998011e-16,-2.422589e-15,2.497514e-15
std,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,...,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00
min,-2.029648e+00,-2.229249e+00,-1.984504e+00,-1.454443e+00,-3.112085e+00,-1.610136e+00,-1.114873e+00,-1.261820e+00,-2.744117e+00,-1.819865e+00,...,-1.726901e+00,-2.223994e+00,-1.693361e+00,-1.222423e+00,-2.682695e+00,-1.443878e+00,-1.305831e+00,-1.745063e+00,-2.160960e+00,-1.601839e+00
25%,-6.893853e-01,-7.259631e-01,-6.919555e-01,-6.671955e-01,-7.109628e-01,-7.470860e-01,-7.437479e-01,-7.379438e-01,-7.032397e-01,-7.226392e-01,...,-6.749213e-01,-7.486293e-01,-6.895783e-01,-6.421359e-01,-6.912304e-01,-6.810833e-01,-7.565142e-01,-7.563999e-01,-6.418637e-01,-6.919118e-01
50%,-2.150816e-01,-1.046362e-01,-2.359800e-01,-2.951869e-01,-3.489108e-02,-2.219405e-01,-3.422399e-01,-3.977212e-01,-7.162650e-02,-1.782793e-01,...,-2.690395e-01,-4.351564e-02,-2.859802e-01,-3.411812e-01,-4.684277e-02,-2.695009e-01,-2.182321e-01,-2.234689e-01,-1.274095e-01,-2.164441e-01
75%,4.693926e-01,5.841756e-01,4.996769e-01,3.635073e-01,6.361990e-01,4.938569e-01,5.260619e-01,6.469351e-01,5.307792e-01,4.709834e-01,...,5.220158e-01,6.583411e-01,5.402790e-01,3.575891e-01,5.975448e-01,5.396688e-01,5.311411e-01,7.125100e-01,4.501382e-01,4.507624e-01
max,3.971288e+00,4.651889e+00,3.976130e+00,5.250529e+00,4.770911e+00,4.568425e+00,4.243589e+00,3.927930e+00,4.484751e+00,4.910919e+00,...,4.094189e+00,3.885905e+00,4.287337e+00,5.930172e+00,3.955374e+00,5.112877e+00,4.700669e+00,2.685877e+00,6.046041e+00,6.846856e+00


In [ ]:
X_mal = X[y == 0]
X_ben = X[y == 1]

# Display the shapes of the resulting subsets
print("Shape of X for malignant:", X_mal.shape)
print("Shape of X for benign:", X_ben.shape)

Shape of X for malignant: (212, 30)
Shape of X for benign: (357, 30)


In [ ]:
mal = 15
total = 300
X_mal_sp = pd.DataFrame(X_mal[:mal, :])
X_ben_sp = pd.DataFrame(X_ben[:total - mal, :])
y_mal_sp = pd.DataFrame(np.ones(mal))
y_ben_sp = pd.DataFrame(np.zeros(total - mal))

In [ ]:
X_sp = pd.concat([X_mal_sp, X_ben_sp])
y_sp = pd.concat([y_mal_sp, y_ben_sp])

In [ ]:
clf = LogisticRegression(solver='lbfgs', max_iter=100).fit(X_sp, np.squeeze(y_sp))

In [ ]:
accuracy_score(y_mal_sp, clf.predict(X_mal_sp))

0.8666666666666667

In [ ]:
accuracy_score(y_ben_sp, clf.predict(X_ben_sp))

1.0

In [ ]:
cm_mal = confusion_matrix(y_mal_sp, clf.predict(X_mal_sp))
cm_mal

array([[ 0,  0],
       [ 2, 13]])

In [ ]:
TPR_mal =  cm_mal.ravel()[3] / (cm_mal.ravel()[3] + cm_mal.ravel()[2] )                               #TPR = TP / P = TP / (TP + FN)
print('TPR is: %.4f' % TPR_mal)

FPR_mal = cm_mal.ravel()[1] / (cm_mal.ravel()[1] + cm_mal.ravel()[0] )                         #FPR = FP / N = FP / (FP + TN)
print('FPR is: %.4f' % FPR_mal)

TPR is: 0.8667
FPR is: nan


<ipython-input-108-0bbe9d53c9e6>:4: RuntimeWarning: invalid value encountered in long_scalars
  FPR_mal = cm_mal.ravel()[1] / (cm_mal.ravel()[1] + cm_mal.ravel()[0] )                         #FPR = FP / N = FP / (FP + TN)


In [ ]:
cm = confusion_matrix(y_sp, clf.predict(X_sp))
cm

array([[285,   0],
       [  2,  13]])

In [ ]:
TPR =  cm.ravel()[3] / (cm.ravel()[3] + cm.ravel()[2] )                               #TPR = TP / P = TP / (TP + FN)
print('TPR is: %.4f' % TPR)

FPR = cm.ravel()[1] / (cm.ravel()[1] + cm.ravel()[0] )                         #FPR = FP / N = FP / (FP + TN)
print('FPR is: %.4f' % FPR)

TPR is: 0.8667
FPR is: 0.0000
